In [ ]:
import pandas as pd
import numpy as np
from unidecode import unidecode

file = pd.ExcelFile('datasets/combustivel-2000a2020.xlsx')

#names = file.sheet_names  # see all sheet names
df_comb = file.parse('Diesel')
df_comb['Combustivel'] = 'Diesel'

for name in ['Etanol', 'Gasolina', 'GNV']:
    df = file.parse(name)
    df['Combustivel'] = name
    df_comb = pd.concat([df_comb, df])

In [ ]:
df_comb

In [ ]:
df_comb.rename(columns={'Município': 'Municipio', 'ESTADO': 'UF'}, inplace = True)
df_comb

In [ ]:
df_comb['Municipio'] = df_comb['Municipio'].convert_dtypes()
df_comb['Municipio'] = df_comb['Municipio'].str.upper()
df_comb['Municipio'] = df_comb['Municipio'].apply(unidecode)
df_comb['UF'] = df_comb['UF'].convert_dtypes()
df_comb['UF'] = df_comb['UF'].str.upper()
df_comb['UF'] = df_comb['UF'].apply(unidecode)

df_comb

In [ ]:
df_rms = pd.read_csv('datasets/rms_rev02.csv')

df_comb_rm = df_comb[df_comb[['UF', 'Municipio']].apply(tuple, axis=1).isin(df_rms[['UF','Municipio']].apply(tuple, axis=1))]
df_comb_rm = pd.merge(df_comb_rm, df_rms[['Municipio', 'UF', 'RM']], on=['Municipio', 'UF'], how='left')

df_comb_rm

In [ ]:
df_comb_rm.dtypes

In [ ]:
df_comb_rm = df_comb_rm.drop(labels=['CÓDIGO IBGE', 'Capital', 'Código RM'], axis=1)
df_comb_rm = df_comb_rm.dropna(axis=1)
df_comb_rm

In [ ]:
df_comb_rm = df_comb_rm.melt(id_vars=['UF', 'Municipio', 'RM', 'Combustivel'], var_name='Ano', value_name='Quantidade')
df_comb_rm

In [ ]:
df_comb_rm = df_comb_rm.reset_index()
df_comb_rm[df_comb_rm['Quantidade'] == 0] = np.NaN
df_comb_rm = df_comb_rm.dropna(subset=['Quantidade'])
df_comb_rm

In [ ]:
df_comb_rm['Combustivel'] = df_comb_rm['Combustivel'].str.upper()
df_comb_rm['Combustivel'] = df_comb_rm['Combustivel'].apply(lambda x: x.replace('ETANOL', 'ALCOOL'))
df_comb_rm_g = df_comb_rm.groupby(['RM', 'Ano', 'Combustivel'])['Quantidade'].sum()
df_comb_rm_g

In [ ]:
df_comb_rm_g.to_csv('datasets/base_volume_comb_rm.csv')